----------------------------------------------------
Modelagem de tópicos de documentos provenientes da base de dados do CPDOC.
----------------------------------------------------
Base de dados: Antonio Azeredo da Silveira.
-----------------------------
Número de documentos: 10.428
-----------------------------
Número de imagens: 66.938
-----------------------------

### Topic: Clustering documents

In [11]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

Specifying the path to the files  

In [12]:
#datapath = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected' # path marcelo
datapath = "../textfiles-corrected-regrouped/"
outputs = "../outputs/"
import os

onlyfiles = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath,f))]

In [13]:
onlyfiles[0:4]

['AAS_mre_rb_1974.04.17_doc_I-66.txt',
 'AAS_mre_onu_1974.03.12_doc_II-49.txt',
 'AAS_mre_onu_1974.03.12_doc_V-2.txt',
 'AAS_mre_be_1974.04.16_doc_VI-18.txt']

#### make list of txt contents of files

In [14]:
count = 0
percentil = int(len(onlyfiles)/100)
textos = []

for txt in onlyfiles:
    with open(os.path.join(datapath,txt), 'r', encoding='iso8859-1') as f:
        count += 1
        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')
        texto = f.read()
        texto = texto.split()
        #print(len(texto))
        symbols = [x for x in string.punctuation]
        texto = [p for p in texto if p not in symbols]
        #print(len(texto))
        texto = [p.strip(string.punctuation) for p in texto]
        #print(len(texto))
        texto = [p for p in texto if not p.isdigit()]
        #print(len(texto))
        texto = [p for p in texto if len(p)>1]
        #print(len(texto))
        #print(texto)
        textos.append(texto)
#print(textos)

100 % done


### Stopwords, stemming, and tokenizing

This section is focused on defining some functions to manipulate the synopses. First, I load [NLTK's](http://www.nltk.org/) list of English stop words. [Stop words](http://en.wikipedia.org/wiki/Stop_words) are words like "a", "the", or "in" which don't convey significant meaning. I'm sure there are much better explanations of this out there.

In [15]:
# load nltk's English stopwords as variable called 'stopwords'
additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words
            
stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [16]:
%time texts = [[word for word in text if word not in stopwords] for text in textos]

CPU times: user 2min 15s, sys: 56 ms, total: 2min 15s
Wall time: 2min 15s


In [17]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

125


In [18]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [19]:
len(corpus)

10550

In [20]:
%time lda = models.LdaModel(corpus, num_topics=40, id2word=dictionary, passes=10)

CPU times: user 54min 22s, sys: 4h 6min 34s, total: 5h 57s
Wall time: 17min 28s


In [46]:
lda.print_topics(-1, num_words=20)

[(0,
  '0.041*ilc + 0.020*watercourses + 0.015*aaó + 0.012*non-navigational + 0.010*attribution + 0.010*chrome + 0.009*colorado + 0.008*nobel + 0.007*nbc + 0.006*confiadas + 0.006*destroyed + 0.006*joins + 0.005*vest + 0.005*perse + 0.004*augurios + 0.004*amlgo + 0.004*profissoes + 0.004*avaí + 0.003*leblon + 0.003*brand'),
 (1,
  '0.011*governo + 0.009*brasil + 0.005*chanceler + 0.005*presidente + 0.004*telegrama + 0.004*excelência + 0.004*argentina + 0.004*assunto + 0.004*brasileiro + 0.003*ter + 0.003*respeito + 0.003*página + 0.003*continuação + 0.003*reunião + 0.003*relações + 0.002*brasileira + 0.002*imprensa + 0.002*embaixador + 0.002*quanto + 0.002*sido'),
 (2,
  '0.142*senegal + 0.088*dacar + 0.051*senghor + 0.018*barre + 0.017*dakar + 0.007*lêopold + 0.007*sêdar + 0.005*feira + 0.004*contingenciamento + 0.003*611145mnex + 0.003*lorenzo + 0.003*barricadas + 0.003*alinentos + 0.003*agréer + 0.003*ivy + 0.003*izabel + 0.002*parecida + 0.002*inaugurando + 0.002*ncm + 0.002*brr'),

In [47]:
lda.show_topics(num_topics=-1)

[(0,
  '0.041*ilc + 0.020*watercourses + 0.015*aaó + 0.012*non-navigational + 0.010*attribution + 0.010*chrome + 0.009*colorado + 0.008*nobel + 0.007*nbc + 0.006*confiadas'),
 (1,
  '0.011*governo + 0.009*brasil + 0.005*chanceler + 0.005*presidente + 0.004*telegrama + 0.004*excelência + 0.004*argentina + 0.004*assunto + 0.004*brasileiro + 0.003*ter'),
 (2,
  '0.142*senegal + 0.088*dacar + 0.051*senghor + 0.018*barre + 0.017*dakar + 0.007*lêopold + 0.007*sêdar + 0.005*feira + 0.004*contingenciamento + 0.003*611145mnex'),
 (3,
  '0.119*rights + 0.096*human + 0.042*racial + 0.036*convention + 0.034*discrimination + 0.029*apartheid + 0.022*social + 0.016*crime + 0.014*forms + 0.013*crimes'),
 (4,
  '0.042*majestade + 0.041*imperial + 0.039*ltd + 0.037*irã + 0.025*princesa + 0.016*alteza + 0.013*nippon + 0.012*keidanren + 0.011*pahlavi + 0.008*macroestrutura'),
 (5,
  '0.025*itaipu + 0.019*corpus + 0.015*rio + 0.014*argentina + 0.011*paraguai + 0.008*cota + 0.006*operação + 0.006*energia + 

In [48]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[34]

(34,
 [('nuclear', 0.059558883938807226),
  ('nucleares', 0.022893285934213612),
  ('energia', 0.022526813529899634),
  ('brasil', 0.014483421412494594),
  ('acordo', 0.014035778846678353),
  ('urânio', 0.012340139687795213),
  ('armas', 0.012121991152178095),
  ('salvaguardas', 0.010541804841425433),
  ('tecnologia', 0.010119802535620793),
  ('aiea', 0.008730021130036696),
  ('cooperação', 0.0080170814266003251),
  ('rfa', 0.0079221448072023162),
  ('tratado', 0.0077881408929063606),
  ('combustível', 0.0076184683897367635),
  ('alemanha', 0.0074525736713479422),
  ('programa', 0.006638155502617888),
  ('federal', 0.0061065217618171842),
  ('internacional', 0.0056453352254054491),
  ('reatores', 0.0051912977962078064),
  ('fins', 0.0051898726287113875)])

Gerando a distribuição de tópicos para um novo documento:

In [50]:
#texts[0]
#corpus[0]
novo_texto = ['nuclear', 'aiea', 'azeredo', 'proliferação','atômica', 'princesa','brasil','alteza','bomba','irã','telegrama','avião']
novo_texto_bow = dictionary.doc2bow(novo_texto)
#print(novo_texto_bow)
print(lda[novo_texto_bow])

[(39, 1), (934, 1), (14066, 1), (30505, 1), (39527, 1), (47622, 1), (53429, 1), (67123, 1), (67506, 1), (90906, 1), (95066, 1), (97138, 1)]
[(4, 0.23187225421431559), (24, 0.082227098424391229), (34, 0.37235995696256796), (36, 0.15745258008265139), (39, 0.088780418008381351)]


Visualizando os tópicos com o PyLDAvis

In [30]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
25     10.626984        1       1  0.211425  0.143518
33     10.220098        1       2 -0.043374 -0.342678
1       8.882384        1       3  0.136203  0.107413
27      7.138496        1       4  0.184882  0.056239
26      6.523365        1       5  0.259469  0.138563
7       5.679010        1       6  0.221542  0.087660
10      5.135480        1       7 -0.018340 -0.346892
35      5.020011        1       8  0.205509  0.113417
20      4.802868        1       9  0.200422 -0.048546
39      4.673267        1      10  0.273431  0.107419
11      4.012956        1      11  0.231972  0.056157
16      3.778932        1      12  0.278820  0.059835
5       3.772569        1      13  0.237814  0.087250
8       3.624326        1      14  0.083089 -0.135028
29      2.274514        1      15  0.093824 -0.019607
31      2.213539        1      16  0.203393 -0.007126
24      2.095615        1      17  0.128506 -0.134983
37      2.094307        1      18  0.215444 -0.019803
6       1.883279        1      19  0.100399 -0.072769
36      1.303602        1      20  0.119796 -0.182176
34      1.179373        1      21  0.153718  0.038224
28      0.920526        1      22 -0.035684 -0.242870
17      0.715375        1      23  0.014614 -0.067853
22      0.259843        1      24 -0.049131 -0.043837
15      0.230960        1      25 -0.094204 -0.048377
3       0.219401        1      26 -0.144829 -0.077743
32      0.177891        1      27 -0.156580 -0.018417
30      0.140075        1      28 -0.183492 -0.010220
13      0.067602        1      29 -0.220670  0.044704
23      0.063807        1      30 -0.201783  0.026854
12      0.039802        1      31 -0.233396  0.068398
18      0.036860        1      32 -0.233601  0.064175
2       0.035391        1      33 -0.238136  0.071389
4       0.031334        1      34 -0.240541  0.074663
14      0.030268        1      35 -0.240033  0.072972
21      0.027159        1      36 -0.240570  0.071802
9       0.019230        1      37 -0.244781  0.082193
0       0.018571        1      38 -0.244585  0.079949
38      0.016256        1      39 -0.245601  0.082919
19      0.014673        1      40 -0.244942  0.083212, topic_info=      Category          Freq            Term         Total  loglift  logprob
term                                                                        
67105  Default  15291.000000       telegrama  15291.000000  30.0000  30.0000
19043  Default  19528.000000          senhor  19528.000000  29.0000  29.0000
18533  Default  21986.000000        ministro  21986.000000  28.0000  28.0000
88634  Default  13151.000000      excelência  13151.000000  27.0000  27.0000
14055  Default  51206.000000          brasil  51206.000000  26.0000  26.0000
95067  Default   7571.000000         nuclear   7571.000000  25.0000  25.0000
39665  Default   9574.000000         general   9574.000000  24.0000  24.0000
27671  Default  22231.000000      presidente  22231.000000  23.0000  23.0000
33871  Default   9234.000000       chanceler   9234.000000  22.0000  22.0000
93239  Default   8667.000000        silveira   8667.000000  21.0000  21.0000
95377  Default  17090.000000      exteriores  17090.000000  20.0000  20.0000
6909   Default   9028.000000          unidos   9028.000000  19.0000  19.0000
3109   Default  24458.000000        relações  24458.000000  18.0000  18.0000
26237  Default  11002.000000          united  11002.000000  17.0000  17.0000
30515  Default   5272.000000          minuta   5272.000000  16.0000  16.0000
82200  Default  11648.000000      embaixador  11648.000000  15.0000  15.0000
90017  Default  12437.000000          acordo  12437.000000  14.0000  14.0000
33382  Default   6964.000000          itaipu   6964.000000  13.0000  13.0000
9774   Default  10256.000000      cooperação  10256.000000  12.0000  12.0000
10204  Default   8087.000000   international   8087.000000  11.0000  11.

In [16]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))

In [17]:
%time hdp = models.HdpModel(corpus, id2word=dictionary)

CPU times: user 7min 4s, sys: 30min 59s, total: 38min 4s
Wall time: 2min 41s


In [18]:
hdp.print_topics()

['topic 0: 0.009*brasil + 0.004*relações + 0.004*governo + 0.004*presidente + 0.004*ministro + 0.003*senhor + 0.003*exteriores + 0.002*telegrama + 0.002*política + 0.002*excelência + 0.002*acordo + 0.002*brasileiro + 0.002*república + 0.002*embaixador + 0.002*desenvolvimento + 0.002*rio + 0.002*cooperação + 0.002*brasileira + 0.002*internacional + 0.002*unidos',
 'topic 1: 0.011*united + 0.008*nations + 0.007*committee + 0.007*international + 0.007*states + 0.007*general + 0.005*assembly + 0.005*resolution + 0.005*economic + 0.004*report + 0.004*development + 0.004*world + 0.003*session + 0.003*republic + 0.003*paragraph + 0.003*meeting + 0.003*secretary-general + 0.003*nuclear + 0.003*developing + 0.002*state',
 'topic 2: 0.007*brasil + 0.005*ministro + 0.004*governo + 0.004*exteriores + 0.004*relações + 0.004*senhor + 0.003*presidente + 0.003*chanceler + 0.003*embaixador + 0.003*telegrama + 0.002*excelência + 0.002*política + 0.002*silveira + 0.002*serie + 0.002*delegação + 0.002*reu

In [19]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(hdp, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      69.755008        1       1  0.317222  0.093117
1      14.557113        1       2  0.285883 -0.380366
2       5.390336        1       3  0.321617  0.079632
8       1.205942        1       4  0.125189  0.047523
11      0.948677        1       5  0.085181  0.049164
7       0.868271        1       6  0.103893  0.076682
5       0.836832        1       7  0.149933  0.086059
4       0.618775        1       8  0.155224  0.050750
13      0.502839        1       9  0.061705 -0.022996
6       0.472058        1      10  0.142495  0.090635
9       0.455759        1      11  0.115907  0.073778
55      0.414342        1      12 -0.029689 -0.009377
3       0.391015        1      13  0.140152 -0.222094
36      0.340259        1      14 -0.017743  0.004146
21      0.257969        1      15  0.023162  0.024223
10      0.223749        1      16  0.130824  0.062267
12      0.192186        1      17  0.057243 -0.125312
17      0.178007        1      18  0.023315  0.026475
53      0.163791        1      19 -0.008938 -0.002573
30      0.154917        1      20  0.000619 -0.009483
15      0.145267        1      21  0.076249  0.052486
16      0.142646        1      22  0.032724  0.020668
35      0.131427        1      23  0.000070 -0.012240
18      0.119711        1      24  0.039350  0.026233
14      0.114851        1      25  0.081429  0.052846
20      0.086193        1      26  0.026606  0.025086
38      0.081767        1      27  0.000237  0.013802
25      0.074584        1      28  0.007919  0.011211
44      0.062220        1      29 -0.005213  0.008147
45      0.061436        1      30 -0.005806 -0.005699
...          ...      ...     ...       ...       ...
93      0.000010        1     121 -0.032347 -0.003987
127     0.000009        1     122 -0.032881 -0.005517
139     0.000008        1     123 -0.033175 -0.003737
129     0.000008        1     124 -0.031891 -0.004747
98      0.000008        1     125 -0.031789 -0.004345
117     0.000008        1     126 -0.032239 -0.003957
132     0.000008        1     127 -0.031760 -0.003220
114     0.000008        1     128 -0.032217 -0.003662
116     0.000008        1     129 -0.032389 -0.005209
118     0.000008        1     130 -0.032449 -0.004359
121     0.000007        1     131 -0.032530 -0.003778
131     0.000007        1     132 -0.031085 -0.003827
108     0.000007        1     133 -0.032593 -0.005255
126     0.000007        1     134 -0.032813 -0.004001
128     0.000007        1     135 -0.031722 -0.005073
113     0.000006        1     136 -0.032270 -0.004927
133     0.000006        1     137 -0.031631 -0.004363
140     0.000006        1     138 -0.032709 -0.004256
99      0.000006        1     139 -0.031856 -0.004901
134     0.000006        1     140 -0.031830 -0.003762
122     0.000006        1     141 -0.032008 -0.003755
142     0.000005        1     142 -0.032482 -0.004523
135     0.000005        1     143 -0.031918 -0.004734
138     0.000004        1     144 -0.032544 -0.004359
145     0.000003        1     145 -0.031750 -0.003451
144     0.000003        1     146 -0.033098 -0.005410
149     0.000003        1     147 -0.032695 -0.004303
143     0.000002        1     148 -0.031947 -0.002915
146     0.000002        1     149 -0.032868 -0.004766
147     0.000002        1     150 -0.032252 -0.004982

[150 rows x 5 columns], topic_info=       Category          Freq             Term         Total  loglift  logprob
term                                                                          
32368   Default  45418.000000           brasil  45418.000000  30.0000  30.0000
25941   Default  21260.000000         relações  21260.000000  29.0000  29.0000
52542   Default  19685.000000         ministro  19685.000000  28.0000  28.0000
50652   Default  17685.000000           senhor  17685.000000  27.0000  27.0000
8514    Default  19761.000000       presidente  19761.000

In [20]:
data_hdpvis = pyLDAvis.gensim.prepare(hdp, corpus, dictionary)
pyLDAvis.save_html(data_hdpvis, os.path.join(outputs,'pyldavis_output_hdp.html'))